rodar colab

In [ ]:
%%writefile dunn_index.cu
#include <stdio.h>
#include <stdlib.h>
#include <float.h>
#include <math.h>
#include <sys/time.h>

#define MAX_POINTS 200
#define MAX_CLUSTERS 10
#define DIM 4

__global__ void pairwise_distances_kernel(float *data, float *distances, int n, int dim) {
    int i = blockIdx.x;
    int j = threadIdx.x;

    if (i < n && j < n) {
        float sum = 0.0f;
        for (int d = 0; d < dim; d++) {
            float diff = data[i * dim + d] - data[j * dim + d];
            sum += diff * diff;
        }
        distances[i * n + j] = sqrtf(sum);
    }
}

float dunn_index_gpu(float *data, int *labels, int n, int dim,
                     int *clusters, int *counts, float *diameters, float *delta_min) {
    float *d_data, *d_distances;
    float *h_distances = (float *)malloc(n * n * sizeof(float));

    // Alocar memória no device
    cudaMalloc((void **)&d_data, n * dim * sizeof(float));
    cudaMalloc((void **)&d_distances, n * n * sizeof(float));
    cudaMemcpy(d_data, data, n * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Executar kernel
    pairwise_distances_kernel<<<n, n>>>(d_data, d_distances, n, dim);
    cudaDeviceSynchronize();
    cudaMemcpy(h_distances, d_distances, n * n * sizeof(float), cudaMemcpyDeviceToHost);

    float min_inter = FLT_MAX;
    float max_intra = 0.0f;

    // Para estatísticas por cluster
    int label_seen[MAX_CLUSTERS] = {0};
    for (int i = 0; i < n; i++) {
        int l = labels[i];
        if (!label_seen[l]) {
            clusters[*counts] = l;
            counts[*counts] = 0;
            diameters[*counts] = 0.0f;
            (*counts)++;
            label_seen[l] = 1;
        }
    }

    // Calcula distâncias
    for (int i = 0; i < n; i++) {
        int li = labels[i];
        for (int j = i + 1; j < n; j++) {
            int lj = labels[j];
            float dist = h_distances[i * n + j];

            if (li != lj && dist < min_inter)
                min_inter = dist;
            if (li == lj && dist > diameters[li])
                diameters[li] = dist;

            if (li == lj && dist > max_intra)
                max_intra = dist;
        }
    }

    if (delta_min) *delta_min = min_inter;

    // Atualiza contagem por cluster
    for (int i = 0; i < n; i++) {
        int l = labels[i];
        for (int c = 0; c < *counts; c++) {
            if (clusters[c] == l) {
                counts[c]++;
                break;
            }
        }
    }

    cudaFree(d_data);
    cudaFree(d_distances);
    free(h_distances);

    return (max_intra == 0) ? 0.0f : min_inter / max_intra;
}

int main() {
    int n = 150;
    int dim = 4;

    float data[MAX_POINTS * DIM];
    int labels[MAX_POINTS];

    FILE *f_data = fopen("iris_data.txt", "r");
    FILE *f_label = fopen("iris_labels.txt", "r");

    if (!f_data || !f_label) {
        printf("Erro ao abrir arquivos.\n");
        return 1;
    }

    for (int i = 0; i < n; i++) {
        if (fscanf(f_label, "%d", &labels[i]) != 1) {
            fprintf(stderr, "Erro ao ler label %d\n", i);
            return 1;
        }
        for (int j = 0; j < dim; j++) {
            if (fscanf(f_data, "%f", &data[i * dim + j]) != 1) {
                fprintf(stderr, "Erro ao ler dado (%d, %d)\n", i, j);
                return 1;
            }
        }
    }
    fclose(f_data);
    fclose(f_label);

    FILE *log = fopen("resultado_benchmark_cuda.tsv", "w");
    fprintf(log, "tamanho\ttempo\tindice_dunn\tdelta_min\n");

    int sizes[] = {30, 60, 90, 120, 150};
    printf("Tamanho\tTempo_CUDA(s)\tDunn_Index\n");

    for (int s = 0; s < 5; s++) {
        int size = sizes[s];

        // Verifica se há ao menos 2 clusters diferentes
        int has_diff = 0;
        for (int i = 0; i < size - 1; i++) {
            for (int j = i + 1; j < size; j++) {
                if (labels[i] != labels[j]) {
                    has_diff = 1;
                    break;
                }
            }
            if (has_diff) break;
        }
        if (!has_diff) continue;

        int clusters[MAX_CLUSTERS] = {0};
        int counts[MAX_CLUSTERS] = {0};
        float diameters[MAX_CLUSTERS] = {0.0f};
        float delta_min = 0.0f;

        struct timeval start, end;
        gettimeofday(&start, NULL);

        float dunn = dunn_index_gpu(data, labels, size, dim, clusters, counts, diameters, &delta_min);

        gettimeofday(&end, NULL);
        double elapsed = (end.tv_sec - start.tv_sec) + ((end.tv_usec - start.tv_usec) / 1e6);

        printf("%d\t%.6f\t\t%.4f\n", size, elapsed, dunn);
        fprintf(log, "%d\t%.6f\t%.4f\t%.4f\n", size, elapsed, dunn, delta_min);

        printf("Clusters:\n");
        for (int i = 0; i < MAX_CLUSTERS && counts[i] > 0; i++) {
            printf("  Label %d: %d pontos, diâmetro = %.4f\n", clusters[i], counts[i], diameters[i]);
        }
        printf("Distância mínima entre clusters = %.4f\n", delta_min);
        printf("--------------------------------------\n");
    }

    fclose(log);
    return 0;
}

In [ ]:
!nvcc -o dunn_index_cuda dunn_index.cu
!./dunn_index_cuda